# Lilly IPO Shard Runner (Colab)

This notebook runs a single shard of `build_ipo_pairs.py` and saves output to Google Drive.

**Before running:**
- Set `SHARD_ID` (00..07).
- Decide how you want to transfer the repo + artifacts to Colab (Git clone or Drive).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Change this to your preferred Drive folder
OUT_DIR = '/content/drive/MyDrive/lilly_ipo'
!mkdir -p "{OUT_DIR}"

In [ ]:
# Persist HF cache to Drive to avoid re-downloading models
import os
HF_CACHE = '/content/drive/MyDrive/hf_cache'
os.makedirs(HF_CACHE, exist_ok=True)
os.environ['HF_HOME'] = HF_CACHE
os.environ['HUGGINGFACE_HUB_CACHE'] = HF_CACHE
os.environ['TRANSFORMERS_CACHE'] = HF_CACHE
print('HF cache set to:', HF_CACHE)
os.environ['HF_HUB_OFFLINE'] = '0'
print('HF_HUB_OFFLINE:', os.environ.get('HF_HUB_OFFLINE'))


In [ ]:
# Optional: pre-download Qwen3-32B into Drive cache
from huggingface_hub import snapshot_download
MODEL_ID = 'Qwen/Qwen3-32B'
snapshot_download(repo_id=MODEL_ID, cache_dir=HF_CACHE, allow_patterns=['*.json','*.safetensors','*.model','*.txt','*.py','*.tiktoken','tokenizer*','merges.txt','vocab.json'])
print('Download complete')


In [ ]:
# Optional: pre-download embedding model into Drive cache
from huggingface_hub import snapshot_download
EMBED_MODEL = 'Qwen/Qwen3-Embedding-0.6B'
snapshot_download(repo_id=EMBED_MODEL, cache_dir=HF_CACHE)
print('Embedding model cached')


In [ ]:
# Download repo (and shards) from GitHub if not already present
import os
REPO_URL = 'https://github.com/rmulligan/lilly-ipo-colab.git'
REPO_DIR = '/content/lilly-ipo-colab'
if not os.path.isdir(REPO_DIR):
    !git clone "{REPO_URL}" "{REPO_DIR}"

print('Repo dir:', REPO_DIR)


In [ ]:
# Toggle 4-bit quantization (set False if bitsandbytes fails)
USE_4BIT = False


In [ ]:
# Toggle classifier usage (set False if embedding model is unavailable)
USE_CLASSIFIER = True


In [ ]:
# Verify adapter path exists
import os
ADAPTER_PATH = '/content/drive/MyDrive/lilly_repo/experiments/lilly_subjective_32b/final_lilly_32b'
print('Adapter exists:', os.path.isdir(ADAPTER_PATH))
print('Adapter contents:')
!ls -la "{ADAPTER_PATH}" | head -n 20


In [ ]:
# Sync adapter from Drive into repo (if missing)
import os, shutil
DRIVE_ADAPTER = '/content/drive/MyDrive/lilly_repo/experiments/lilly_subjective_32b/final_lilly_32b'
REPO_DIR = '/content/lilly-ipo-colab'
REPO_ADAPTER = REPO_DIR + '/experiments/lilly_subjective_32b/final_lilly_32b'
if os.path.isdir(DRIVE_ADAPTER):
    os.makedirs(os.path.dirname(REPO_ADAPTER), exist_ok=True)
    if not os.path.isdir(REPO_ADAPTER):
        print('Copying adapter into repo...')
        shutil.copytree(DRIVE_ADAPTER, REPO_ADAPTER)
    else:
        print('Repo adapter already exists, skipping copy.')
else:
    print('Drive adapter not found; please check path:', DRIVE_ADAPTER)


In [ ]:
# Verify selected shard exists
import os
SHARD_ID = '00'  # set before running
SHARD_PATH = f'/content/lilly-ipo-colab/experiments/affect_ipo/shards/prompts_shard_{SHARD_ID}.json'
print('Shard path:', SHARD_PATH)
print('Shard exists:', os.path.isfile(SHARD_PATH))
!ls -la "{SHARD_PATH}"


In [ ]:
# Check required file paths before running
import os
REQ_PATHS = [
    f'{REPO_DIR}/scripts/build_ipo_pairs.py',
    f'{REPO_DIR}/experiments/affect_ipo/shards/prompts_shard_{SHARD_ID}.json',
    f'{REPO_DIR}/experiments/egist_32b/unified_lilly_probe.pt',
    f'{REPO_DIR}/experiments/egist_32b/unified_dimensions.json',
    f'{REPO_DIR}/experiments/lilly_subjective_32b/affect_classifier.pt',
    f'{REPO_DIR}/experiments/lilly_subjective_32b/final_lilly_32b',
]
for p in REQ_PATHS:
    print(p, '->', 'OK' if os.path.exists(p) else 'MISSING')


In [ ]:
# Run shard and save to Drive with timestamp
import time
ts = time.strftime('%Y%m%d_%H%M%S')
OUT_PATH = f'/content/drive/MyDrive/lilly_ipo/pairs_shard_{SHARD_ID}_{ts}.jsonl'
print('Writing to:', OUT_PATH)
CMD = f"python {REPO_DIR}/scripts/build_ipo_pairs.py \\
  --input {REPO_DIR}/experiments/affect_ipo/shards/prompts_shard_{SHARD_ID}.json \\
  --output {OUT_PATH} \\
  --adapter-path {REPO_DIR}/experiments/lilly_subjective_32b/final_lilly_32b \\
  --probe-path {REPO_DIR}/experiments/egist_32b/unified_lilly_probe.pt \\
  --dimensions-path {REPO_DIR}/experiments/egist_32b/unified_dimensions.json \\
  --classifier-path {REPO_DIR}/experiments/lilly_subjective_32b/affect_classifier.pt \\
  --num-candidates 4 \\
  --max-new-tokens 160 \\
  --max-samples 100"
if USE_4BIT:
    CMD += ' --use-4bit'
if not USE_CLASSIFIER:
    CMD += ' --no-classifier'
!{CMD}


In [ ]:
# Optional: run all shards sequentially in one session
import time
ALL_SHARDS = [f'{i:02d}' for i in range(8)]
for sid in ALL_SHARDS:
    ts = time.strftime('%Y%m%d_%H%M%S')
    out_path = f'/content/drive/MyDrive/lilly_ipo/pairs_shard_{sid}_{ts}.jsonl'
    print('Running shard', sid, '->', out_path)
    CMD = f"python {REPO_DIR}/scripts/build_ipo_pairs.py \\
      --input {REPO_DIR}/experiments/affect_ipo/shards/prompts_shard_{sid}.json \\
      --output {out_path} \\
      --adapter-path {REPO_DIR}/experiments/lilly_subjective_32b/final_lilly_32b \\
      --probe-path {REPO_DIR}/experiments/egist_32b/unified_lilly_probe.pt \\
      --dimensions-path {REPO_DIR}/experiments/egist_32b/unified_dimensions.json \\
      --classifier-path {REPO_DIR}/experiments/lilly_subjective_32b/affect_classifier.pt \\
      --num-candidates 4 \\
      --max-new-tokens 160 \\
      --max-samples 100"
    if USE_4BIT:
        CMD += ' --use-4bit'
if not USE_CLASSIFIER:
    CMD += ' --no-classifier'
    !{CMD}


## Option A: Git clone repo
If your repo is accessible via git (public or with credentials), clone it below.
Otherwise use Option B to upload a zip or use Drive.

In [ ]:
# REPLACE with your repo URL
REPO_URL = 'YOUR_REPO_URL_HERE'
REPO_DIR = '/content/lilly'
# Uncomment if using git
# !git clone "{REPO_URL}" "{REPO_DIR}"

## Option B: Use Drive to provide artifacts
Place your repo or just the needed files in Drive and set `REPO_DIR` accordingly.
Required files:
- scripts/build_ipo_pairs.py
- experiments/affect_ipo/shards/prompts_shard_XX.json
- experiments/egist_32b/unified_lilly_probe.pt
- experiments/egist_32b/unified_dimensions.json
- experiments/lilly_subjective_32b/affect_classifier.pt
- experiments/lilly_subjective_32b/final_lilly_32b (adapter dir)
- Adapter already uploaded to Drive at: `/content/drive/MyDrive/lilly_repo/experiments/lilly_subjective_32b/final_lilly_32b`


In [ ]:
# If using Drive, set REPO_DIR to the folder containing the repo/artifacts
REPO_DIR = '/content/drive/MyDrive/lilly_repo'  # change as needed

In [ ]:
# Install deps (requires runtime restart after install)
!pip -q install -U transformers accelerate peft "bitsandbytes>=0.46.1"

import bitsandbytes as bnb
print('bitsandbytes version:', getattr(bnb, '__version__', 'unknown'))
print('If version < 0.46.1, restart runtime and re-run this cell.')


**Note:** After installing/upgrading `bitsandbytes`, use `Runtime → Restart runtime` in Colab, then re-run the notebook from the top.

## (Optional) Hugging Face login
Only needed if you use gated/private models.

In [ ]:
# from huggingface_hub import login
# login()

In [ ]:
# Merge all shard outputs on Drive into a single JSONL
import glob, os
from pathlib import Path
OUT_DIR = Path('/content/drive/MyDrive/lilly_ipo')
files = sorted(OUT_DIR.glob('pairs_shard_*.jsonl'))
print('Found', len(files), 'shard files')
merged_path = OUT_DIR / 'pairs_merged.jsonl'
with merged_path.open('w') as out:
    for f in files:
        for line in f.read_text().splitlines():
            if line.strip():
                out.write(line + '\n')
print('Merged ->', merged_path)


## Download merged output
In Colab, you can download the merged file directly:
```python
from google.colab import files
files.download('/content/drive/MyDrive/lilly_ipo/pairs_merged.jsonl')
```
Or sync it from Drive to your local machine.


## Done
Output will be saved to Google Drive at:
`/content/drive/MyDrive/lilly_ipo/pairs_shard_XX.jsonl`